In [3]:
import pandas as pd
import datetime
import numpy as np
import re
import sys
sys.path.append("..")
from aoc18.utils import read_input


# Day 13

In [88]:
inp = read_input(13, root_path="..")
inp_test = read_input(13, root_path="..", test=True)

In [78]:
straight = np.array([
    [1, 0],
    [0, 1]])

left = np.array([
    [ 0, 1],
    [-1, 0]])

right = np.array([
    [0, -1],
    [1,  0]])

anti_slash = np.array([
    [0, 1],
    [1, 0]])

slash = np.array([
   [ 0, -1],
   [-1,  0]])

def build_action(move_array):
    return np.vstack([
        np.hstack([np.identity(2), np.zeros((2,2))]),
        np.hstack([move_array]*2)
    ])

milestone_mapping = {
    "-": build_action(straight),
    "|": build_action(straight),
    "\\": build_action(anti_slash),
    "/": build_action(slash),
    ">": build_action(straight),
    "<": build_action(straight),
    "^": build_action(straight),
    "v": build_action(straight),
    "+": "intersection"
}

carts_mapping = {
    ">": np.array([ 0,  1]),
    "<": np.array([ 0, -1]),
    "v": np.array([ 1,  0]),
    "^": np.array([-1,  0])
}

intersections = [build_action(left), build_action(straight), build_action(right)]


class Milestone:
    
    def __init__(self, string):
        self.string = string
        self.action = self.get_action(string)
        
    def get_action(self, car):
        return milestone_mapping[car]
    
    def __str__(self):
        return self.string
    
    def __repr__(self):
        return "Milestone(%s)" % self.string

In [134]:
def get_collisions(carts_map):
    collisions = []
    for i in range(len(carts_map) - 1):
        for j in range(i + 1, len(carts_map)):
            if np.array_equal(carts_map[i]["cart"][:2], carts_map[j]["cart"][:2]):
                collisions.append((i,j))
    return collisions

def check_collision(carts_map):
    return len(get_collisions(carts_map))>0

def sort_carts(carts_map):
    return sorted(carts_map.items(), key=lambda kv: (kv[1]["cart"][0], kv[1]["cart"][1]))

In [135]:
def get_initial_state(inp):
    this_map = list(map(lambda i: list(map(lambda i: " ", range(len(inp[0][:-1])))), range(len(inp))))
    carts_map = {}
    cart_id = 0
    carts_order = []
    for x, line in enumerate(inp):
        for y, car in enumerate(line[:-1]):
            if car != " ":
                this_map[x][y] = Milestone(car)
            if car in carts_mapping:
                carts_map[cart_id] = {
                    "cart": np.hstack([np.array([x, y]), carts_mapping[car]]),
                    "status": 0
                }
                carts_order.append(cart_id)
                cart_id += 1
    return this_map, carts_map, carts_order

In [137]:
def run_one_step(this_map, carts_map, carts_order):
    for cart_id, cart in sort_carts(carts_map):
        milestone = this_map[cart["cart"][0]][cart["cart"][1]]
        if milestone.string == "+":
            action = intersections[np.mod(cart["status"], 3)]
            carts_map[cart_id]["status"] += 1
        else:
            action = milestone.action
        carts_map[cart_id]["cart"] = cart["cart"].dot(action).astype(int)
    return this_map, carts_map, carts_order

In [138]:
def run1(inp):
    this_map, carts_map, carts_order = get_initial_state(inp)
    iteration = 0
    while True:
        this_map, carts_map, carts_order = run_one_step(this_map, carts_map, carts_order)
        iteration += 1
        collisions = get_collisions(carts_map)
        if len(collisions) > 0:
            break
    cart_id_1, cart_id_2 = collisions[0]
    print("Collision after %d iterations! Cart %d and %d crashed at point (%d, %d)" % (
        iteration,
        cart_id_1,
        cart_id_2,
        carts_map[cart_id_1]["cart"][1],
        carts_map[cart_id_1]["cart"][0]
        )
    )
    return (carts_map[cart_id_1]["cart"][1], carts_map[cart_id_1]["cart"][0])
        


In [140]:
run1(inp)

Collision after 174 iterations! Cart 11 and 16 crashed at point (64, 86)


(64, 86)

# Day 17

In [4]:
inp = read_input(17, root_path="..")
inp_test = read_input(17, root_path="..", test=True)

In [122]:
def parse_line(line):
    single, multiples = line.strip().split(", ")
    xoy = int(single.split("=")[-1])
    minxy, maxxy = map(int, multiples.split("=")[-1].split(".."))
    if single[0] == "x":
        return map(lambda y: (xoy, y), range(minxy, maxxy+1))
    else:
        return map(lambda x: (x, xoy), range(minxy, maxxy+1))

def parse_inp(inp):
    return set([xy for this_range in map(parse_line, inp) for xy in this_range])


class Ground:
    
    def __init__(self, source):
        self.source = source
        self.mat = None
        self.xmin, self.ymin, self.xmax, self.ymax = [0]*4
        
    @staticmethod
    def _to_str(element):
        mapping = dict([(0, "."), (1, "#"), (-1, "|"), (-2, "~")])
        return mapping[element]
        
    def set(self, inp):
        this_range = parse_inp(inp)
        x_s = list(map(lambda xy: xy[0], this_range))
        y_s = list(map(lambda xy: xy[1], this_range))
        self.xmin, self.ymin, self.xmax, self.ymax = min(x_s), min(y_s), max(x_s), max(y_s)
        self.mat = np.zeros((self.ymax+1, self.xmax+2-self.xmin))
        for x, y in this_range:
            self.mat[y, (x-self.xmin)+1] = 1
            
    def get_element(self, x, y):
        return self.mat[y, (x-self.xmin)+1]
    
    def wet(self, position):
        x, y = position
        self.mat[y, (x-self.xmin) + 1] = -1
    
    def __str__(self):
        if self.mat is not None:
            return "\n".join(map(lambda line: "".join(map(self._to_str, line)), self.mat))
        
    def dump(self, file_path):
        with open(file_path, "w") as groundfile:
            groundfile.write("%s" % self)
            
    def count_wet_tiles(self):
        sub_mat = self.mat[self.ymin:, :]
        return (sub_mat == -1).sum()

In [181]:
ground_test = Ground((500,0))
ground_test.set(inp_test)
ground_test.xmin, ground_test.ymin, ground_test.xmax, ground_test.ymax

(495, 1, 506, 13)

In [182]:
ground = Ground((500,0))
ground.set(inp)
ground.xmin, ground.ymin, ground.xmax, ground.ymax

(108, 3, 537, 1913)

In [183]:
def fall(ground, position):
    y_down = position[1] + 1
    while y_down <= ground.ymax and ground.get_element(position[0], y_down) == 0:
        ground.wet((position[0], y_down))
        y_down += 1
    return position[0], y_down - 1

def spread_side(ground, position, left_or_right):
    x_side = position[0] + left_or_right
    while True:
        new_position = ground.get_element(x_side, position[1])
        down = ground.get_element(x_side, position[1] + 1)
        if new_position == 1:
            status = "blocked"
            break
        ground.wet((x_side, position[1]))
        if down == 0:
            status = "fall"
            break
        x_side += left_or_right
    return status, (x_side)

In [184]:
def fall_from(position, ground):
    bottom_position = fall(ground, position)
    if bottom_position[1] < ground.ymax:
        left_status, left_x = spread_side(ground, bottom_position, -1)
        right_status, right_x = spread_side(ground, bottom_position, 1)
        while (left_status, right_status) == ('blocked', 'blocked'):
            bottom_position = bottom_position[0], bottom_position[1] - 1
            left_status, left_x = spread_side(ground, bottom_position, -1)
            right_status, right_x = spread_side(ground, bottom_position, 1)
        if left_status == 'fall':
            fall_from((left_x, bottom_position[1]), ground)
        if right_status == 'fall':
            fall_from((right_x, bottom_position[1]), ground)
    else:
        print("touched down at position (%d, %d)" % bottom_position)

In [185]:
fall_from((500, 0), ground_test)

touched down at position (497, 13)
touched down at position (505, 13)


In [186]:
print(ground_test)

.............
......|.....#
.#..#||||...#
.#..#||#|....
.#..#||#|....
.#|||||#|....
.#|||||#|....
.#######|....
........|....
...|||||||||.
...|#|||||#|.
...|#|||||#|.
...|#|||||#|.
...|#######|.


In [187]:
ground_test.count_wet_tiles()

57

In [188]:
ground.dump('../data/17_ground.txt')

In [189]:
fall_from((500, 0), ground)

touched down at position (107, 1913)
touched down at position (121, 1913)
touched down at position (106, 1913)
touched down at position (105, 1913)
touched down at position (120, 1913)
touched down at position (119, 1913)
touched down at position (104, 1913)
touched down at position (103, 1913)
touched down at position (118, 1913)
touched down at position (146, 1913)
touched down at position (117, 1913)
touched down at position (147, 1913)
touched down at position (116, 1913)
touched down at position (148, 1913)
touched down at position (115, 1913)
touched down at position (149, 1913)
touched down at position (152, 1913)
touched down at position (102, 1913)
touched down at position (114, 1913)
touched down at position (150, 1913)
touched down at position (113, 1913)
touched down at position (151, 1913)
touched down at position (153, 1913)
touched down at position (101, 1913)
touched down at position (100, 1913)
touched down at position (154, 1913)
touched down at position (99, 1913)
to

touched down at position (366, 1913)
touched down at position (401, 1913)
touched down at position (-29, 1913)
touched down at position (-28, 1913)
touched down at position (-64, 1913)
touched down at position (-27, 1913)
touched down at position (-63, 1913)
touched down at position (-26, 1913)
touched down at position (-191, 1913)
touched down at position (-25, 1913)
touched down at position (-199, 1913)
touched down at position (-24, 1913)
touched down at position (-200, 1913)
touched down at position (-23, 1913)
touched down at position (-201, 1913)
touched down at position (-202, 1913)
touched down at position (-22, 1913)
touched down at position (-203, 1913)
touched down at position (-21, 1913)
touched down at position (-204, 1913)
touched down at position (-20, 1913)
touched down at position (-205, 1913)
touched down at position (-19, 1913)
touched down at position (-206, 1913)
touched down at position (-18, 1913)
touched down at position (-207, 1913)


IndexError: index -432 is out of bounds for axis 1 with size 431

In [190]:
ground.dump('../data/17_ground_wet.txt')

In [131]:
ground.count_wet_tiles()

108785

In [66]:
level_up_position = level_up_position[0], level_up_position[1] - 1
left = spread_side(ground_test, level_up_position, -1)
right = spread_side(ground_test, level_up_position, 1)
print(left, right)

('blocked', 498) ('blocked', 501)


In [67]:
level_up_position = level_up_position[0], level_up_position[1] - 1
left = spread_side(ground_test, level_up_position, -1)
right = spread_side(ground_test, level_up_position, 1)
print(left, right)

('blocked', 498) ('blocked', 501)


In [68]:
level_up_position = level_up_position[0], level_up_position[1] - 1
left = spread_side(ground_test, level_up_position, -1)
right = spread_side(ground_test, level_up_position, 1)
print(left, right)

('blocked', 498) ('fall', 502)


In [69]:
ground_position = fall(ground_test, (right[1], level_up_position[1]))
ground_position

(502, 12)

In [59]:
ground.dump("../data/17_ground.txt")